# Modello originale

In [1]:
import torch
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from data_utils import mnist_preparation, add_salt_and_pepper_noise 
from evaluationUtils import calculate_mcm_accuracy
from tqdm import tqdm
import matplotlib as plt
from mcmadaptablemodel import MCMQuantumModel, MCMCircuit
from bitflipmodel import flipQuantumCircuit, flipFullQuantumModel
from pennylane import Device
from pennylane.measurements import StateMP
from torch.nn import Module, ParameterDict
import matplotlib.pyplot as plt
from OriginalModel import FullQuantumModel, QuantumCircuit
import warnings
from typing import Optional, Dict, List, Any
from torch.utils.data import DataLoader, dataloader
from time import time
import math
from pennylane.measurements import MidMeasureMP
torch.manual_seed(1234)

In [2]:
labels = [0,1,2,3]
# Download MNIST and prepare transforms
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
#train/vali/test 70/15/15 split, see data_utils.py for further details
train_dataloader, validation_dataloader, test_dataloader = mnist_preparation(dataset=mnist, labels = labels, train_test_ratio=0.7,batch_size=3713, vali_test_ratio=0.5)

print("Images in the training set: ", len(train_dataloader.dataset), "\n Images in the validation set: ", len(validation_dataloader.dataset), "\n Images in the test set: ", len(test_dataloader.dataset))

In [3]:
import pickle
with open('/Users/jackvittori/Desktop/bitflip/om/trhistory.pickle', "rb") as file: 
    dict = pickle.load(file)

In [30]:
dev = qml.device("default.mixed", wires=8)
@qml.qnode(dev)
def quantum_function(params: Dict, prob: float, state: torch.Tensor = None):
    measurement = []
    if state is not None:
        # state vector initialization with input
        qml.QubitStateVector(state, wires=range(8))

    for i in range(8):
        for j in range(8):
            qml.RX(params[f'layer_{i}'][j, 0], wires=j)
            qml.RY(params[f'layer_{i}'][j, 1], wires=j)
            qml.RZ(params[f'layer_{i}'][j, 2], wires=j)
        for j in range(8):
            qml.CNOT(wires=[j, (j + 1) % 8])
            qml.BitFlip(p=prob, wires=(j + 1) % 8)   
    return qml.probs()

In [31]:
probs_distribution_no = []

for img, target in tqdm(test_dataloader.dataset):
    img = img / torch.linalg.norm(img).view(-1, 1)
    probs = quantum_function(params = dict['weights'], prob=0.001, state=img)
    class_probabilities = torch.zeros(4, dtype=torch.float32)
    for idx in range(4):
    # Calculate the index range for each class
        start_idx = int(idx * (256 / 4))
        end_idx = int((idx + 1) * (256 / 4))
        class_probabilities[idx] = torch.sum(probs[start_idx:end_idx])
        
    prediction = torch.argmax(class_probabilities)
    confidence = class_probabilities[prediction]
    probs_distribution_no.append(confidence.item())

In [32]:
probs_distribution_no

In [33]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(10, 5))

# Creazione dell'istogramma per il terzo array con maggiore trasparenza
ax.hist(probs_distribution_no, bins=60, density=False, alpha=0.2, color='green', edgecolor='black', label='Original model probability distribution')

# Titolo e etichette degli assi
ax.set_title('Prediction confidence bit flip original model', fontsize=16)
ax.set_xlabel('Prediction confidence', fontsize=14)
ax.set_ylabel('Occurrencies', fontsize=14)

# Imposta i tick dell'asse X con intervalli di 0.05 da 0.25 a 0.6
ax.set_xticks(np.arange(0.25, 0.65, 0.05))

# Aggiunta della legenda per distinguere le distribuzioni
ax.legend()

# Griglia e layout
ax.grid(True)
plt.tight_layout()

# Salvataggio del grafico
plt.savefig('/Users/jackvittori/Desktop/bitflip/om/bitflip001.png', dpi=300)
plt.show()